# DALI Expressions and Arithmetic Operators

This example shows you how to use binary arithmetic operators in the DALI Pipeline that allow for element-wise operations on tensors in a pipeline. We will provide information about the available operators and examples of using constant and scalar inputs.

## Supported Operators

DALI currently supports the following operators: 
 - Unary arithmetic operators: `+`, `-`;
 - Binary arithmetic operators: `+`, `-`, `*`, `/`, and `//`;
 - Comparison operators: `==`, `!=`, `<`, `<=`, `>`, `>=`;
 - Bitwise binary operators: `&`, `|`, `^`.
 
Binary operators can be used as an operation between two tensors, between a tensor and a scalar or a tensor and a constant. By tensor, we consider the output of DALI operators (regular or other arithmetic operators). Unary operators work only with tensor inputs.

In this section we focus on binary arithmetic operators, Tensor, Constant and Scalar operands. The detailed type promotion rules for comparison and bitwise operators are covered in the **Supported operations** section and other examples.

### Prepare the Test Pipeline

1. Prepare the helper code, so we can easily manipulate the types and values that will appear as tensors in the DALI pipeline.

2. We will be using numpy as source for the custom provided data, so we need to import several things from DALI needed to create the Pipeline and use the ExternalSource operator. 

In [1]:
import numpy as np
from nvidia.dali.pipeline import Pipeline
import nvidia.dali.fn as fn           
import nvidia.dali.types as types
from nvidia.dali.types import Constant

### Defining the Data

To define the data, because there are binary operators, two inputs are required. 
We will create a simple helper function that returns two batches of hardcoded data, that are stored as `np.int32`. In an actual scenario the data that is processed by the DALI arithmetic operators would be tensors produced by another operator that contains some images, video sequences or other data.

You can experiment by changing those values or adjusting the `get_data()` function to use different input data. 

**Note**: The shapes of both inputs need to match since operations are performed element-wise. 

In [2]:
left_magic_values = [
    [[42, 7, 0], [0, 0, 0]], 
    [[5, 10, 15], [10, 100, 1000]]
]

right_magic_values = [
    [[3, 3, 3], [1, 3, 5]], 
    [[1, 5, 5], [1, 1, 1]]
]

batch_size = len(left_magic_values)

def convert_batch(batch):
    return [np.int32(tensor) for tensor in batch]

def get_data():
    return (convert_batch(left_magic_values), convert_batch(right_magic_values))

## Operating on Tensors

### Defining the Pipeline

1. To define the pipeline, the data will be obtained from the `get_data` function and made available to the pipeline through `ExternalSource`.

**Note**: You do not need to instantiate any additional operators, we can use regular Python arithmetic expressions on the results of other operators.

2. You can manipulate the source data by adding, multiplying and dividing it.

In [3]:
pipe = Pipeline(batch_size=batch_size, num_threads=4, device_id=0)
with pipe:
    l, r = fn.external_source(source=get_data, num_outputs=2)
    sum_result = l + r
    mul_result = l * r
    div_result = l // r
    pipe.set_outputs(l, r, sum_result, mul_result, div_result)

### Running the Pipeline

3. Build and run our pipeline

In [4]:
pipe.build()
out = pipe.run()

4. Display the results:

In [5]:
def examine_output(pipe_out):
    l = pipe_out[0].as_array()
    r = pipe_out[1].as_array()
    sum_out = pipe_out[2].as_array()
    mul_out = pipe_out[3].as_array()
    div_out = pipe_out[4].as_array()
    print("{}\n+\n{}\n=\n{}\n\n".format(l, r, sum_out))
    print("{}\n*\n{}\n=\n{}\n\n".format(l, r, mul_out))
    print("{}\n//\n{}\n=\n{}\n\n".format(l, r, div_out))

examine_output(out)

[[[  42    7    0]
  [   0    0    0]]

 [[   5   10   15]
  [  10  100 1000]]]
+
[[[3 3 3]
  [1 3 5]]

 [[1 5 5]
  [1 1 1]]]
=
[[[  45   10    3]
  [   1    3    5]]

 [[   6   15   20]
  [  11  101 1001]]]


[[[  42    7    0]
  [   0    0    0]]

 [[   5   10   15]
  [  10  100 1000]]]
*
[[[3 3 3]
  [1 3 5]]

 [[1 5 5]
  [1 1 1]]]
=
[[[ 126   21    0]
  [   0    0    0]]

 [[   5   50   75]
  [  10  100 1000]]]


[[[  42    7    0]
  [   0    0    0]]

 [[   5   10   15]
  [  10  100 1000]]]
//
[[[3 3 3]
  [1 3 5]]

 [[1 5 5]
  [1 1 1]]]
=
[[[  14    2    0]
  [   0    0    0]]

 [[   5    2    3]
  [  10  100 1000]]]




The resulting tensors are obtained by applying the arithmetic operation between corresponding elements of its inputs.

With an exception for scalar tensor inputs that we will describe in the next section, the shapes of the arguments should match the arithmetic operators, otherwise you will get an error. 

## Constant and Scalar Operands
Until now we considered only tensor inputs of matching shapes for inputs of arithmetic operators. DALI allows one of the operands to be a constant or a batch of scalars, and such operands can appear on both sides of binary expressions.

## Constants
The constant operand for arithmetic operator can be one of the following options: 

 - Values of Python's `int` and `float` types that are used directly.
 
 - Values that are wrapped in `nvidia.dali.types.Constant`.
 
The operation between the tensor and the constant results in the constant that is broadcast to all tensor elements.

**Note**: Currently, the values of the integral constants are passed internally to DALI as int32 and the values of floating point constants are passed to DALI as float32.

With regard to type promotion, the Python `int` values will be treated as `int32` and the `float` as `float32`.

The DALI `Constant` can be used to indicate other types. It accepts `DALIDataType` enum values as second argument and has convenience member functions such as `.uint8()` or `.float32()` that can be used for conversions.


### Using the Constants

1. Adjust the Pipeline to utilize constants. 

In [6]:
pipe = Pipeline(batch_size=batch_size, num_threads=4, device_id=0)
with pipe:
    l, r = fn.external_source(source=get_data, num_outputs=2)
    add_200 = l + 200
    mul_075 = l * 0.75
    sub_15 = Constant(15).float32() - r
    pipe.set_outputs(l, r, add_200, mul_075, sub_15)

In [7]:
pipe.build()
out = pipe.run()

2. Display the results:

In [8]:
def examine_output(pipe_out):
    l = pipe_out[0].as_array()
    r = pipe_out[1].as_array()
    add_200 = pipe_out[2].as_array()
    mul_075 = pipe_out[3].as_array()
    sub_15 = pipe_out[4].as_array()
    print("{}\n+ 200 =\n{}\n\n".format(l, add_200))
    print("{}\n* 0.75 =\n{}\n\n".format(l, mul_075))
    print("15 -\n{}\n=\n{}\n\n".format(r, sub_15))

examine_output(out)

[[[  42    7    0]
  [   0    0    0]]

 [[   5   10   15]
  [  10  100 1000]]]
+ 200 =
[[[ 242  207  200]
  [ 200  200  200]]

 [[ 205  210  215]
  [ 210  300 1200]]]


[[[  42    7    0]
  [   0    0    0]]

 [[   5   10   15]
  [  10  100 1000]]]
* 0.75 =
[[[ 31.5    5.25   0.  ]
  [  0.     0.     0.  ]]

 [[  3.75   7.5   11.25]
  [  7.5   75.   750.  ]]]


15 -
[[[3 3 3]
  [1 3 5]]

 [[1 5 5]
  [1 1 1]]]
=
[[[12. 12. 12.]
  [14. 12. 10.]]

 [[14. 10. 10.]
  [14. 14. 14.]]]




The constant value is used with all elements of all tensors in the batch.

## Dynamic Scalars

It is sometimes useful to evaluate an expression with one argument being a tensor, and the other argument being scalar. If the scalar value is constant during the execution of the pipeline, `types.Constant` can be used. When dynamic scalar values are needed, they can be constructed as 0D tensors with empty shape. If DALI encounters this tensor type, DALI will broadcast it to match the shape of the other tensor argument.

**Note**: DALI operates on batches. The scalars are also supplied as batches, with each scalar operand being used with other operands at the same index in the batch.

### Using Scalar Tensors

1. Use an `ExternalSource` to generate a sequence of numbers which will be added to the tensor operands.

In [9]:
pipe = Pipeline(batch_size=batch_size, num_threads=4, device_id=0)
with pipe:
    tensors = fn.external_source(lambda: get_data()[0])
    scalars = fn.external_source(lambda: np.arange(1, batch_size + 1))
    pipe.set_outputs(tensors, scalars, tensors + scalars)

2. Build and run the Pipeline. 

This process allows you to scale your input by some random numbers that were generated by the `Uniform` Operator.

In [10]:
pipe.build()
out = pipe.run()

In [11]:
def examine_output(pipe_out):
    t = pipe_out[0].as_array()
    uni = pipe_out[1].as_array()
    scaled = pipe_out[2].as_array()
    print("{}\n+\n{}\n=\n{}".format(t, uni, scaled))

examine_output(out)

[[[  42    7    0]
  [   0    0    0]]

 [[   5   10   15]
  [  10  100 1000]]]
+
[1 2]
=
[[[  43    8    1]
  [   1    1    1]]

 [[   7   12   17]
  [  12  102 1002]]]


The first scalar in the batch (1) is added to all elements in the first tensor, and the second scalar (2) is added to the second tensor.